In [ ]:
import numpy as np
import pandas as pd
import tensorflow.keras
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import L2 as l2
from tensorflow.keras.layers import Activation,ZeroPadding2D,AveragePooling2D,Dense,Flatten,Add,Dropout,Conv2D,Input,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
train_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
test_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
labels=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
labels

In [ ]:
# sample=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
# sample

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data


In [ ]:
# path0='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-1.dcm'
# img1=load_dicom(path0)
# img2=cv2.resize(img1,(224,224))
# print(img1.shape)
# print(img2.shape) 

In [ ]:
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
trainset=[]
trainlabel=[]
trainidt=[]
for i in tqdm(range(len(labels))):
    idt=labels.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:] 
    path=os.path.join(train_dir,idt2,'T1wCE')              
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        trainset+=[image]
        trainlabel+=[labels.loc[i,'MGMT_value']]
        trainidt+=[idt]
    


In [ ]:
test_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
testset=[]
testidt=[]
for i in tqdm(range(len(sample))):
    idt=sample.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:]
    path=os.path.join(test_dir,idt2,'T1wCE')               
    for im in os.listdir(path):   
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        testset+=[image]
        testidt+=[idt]

#1 2 3 4 5
#->10000 01000 00100 00010#one hot encoding
        
from tensorflow.keras.utils import to_categorical
y0=np.array(trainlabel)
Y_train=to_categorical(y0)
X_train=np.array(trainset)
X_test=np.array(testset)


In [ ]:
model = Sequential()
model.add(Conv2D(64,(4,4),input_shape = (9,9,1),activation = 'relu'))
model.add(Conv2D(32,(2,2),activation = 'relu'))
model.add(Conv2D(64,(2,2),activation = 'relu'))
model.add(Conv2D(32,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit(X_train[500:],Y_train[500:],epochs=100) #trains the model on train dataset

In [ ]:
PathtoPickle1 = "../input/picklefiles/X_trainfull_FLAIR.pickle"
PathtoPickle2 = "../input/picklefiles/Y_trainfull_FLAIR.pickle"
import pickle
with open(PathtoPickle1, 'rb') as f:
    X_train=pickle.load(f)
    f.close()

with open(PathtoPickle2, 'rb') as f:
    Y_train=pickle.load(f)
    f.close()

In [ ]:
Predicted = FlairModel.predict(X_train[0:500])
TrueValues=0
for i in range(0,500):
    if(np.argmax(Predicted[i])==np.argmax(Y_train[i])):
        TrueValues+=1
print(TrueValues/500)

In [ ]:
model.save('./model_T1wCE.h5')

In [ ]:
model.load_weights("../input/model-tiwceh5/model_t1wce.h5")

# End of the Code

In [ ]:
# Predicted[0:100]

In [ ]:
# trainlabel[0:1000]

In [ ]:
# Y_train[0:100]

In [ ]:
import pickle
with open('X_trainfull_FLAIR.pickle', 'wb') as f:
    pickle.dump(X_train, f)
    f.close()

In [ ]:
import pickle
with open('Y_trainfull_FLAIR.pickle', 'wb') as f:
    pickle.dump(Y_train, f)
    f.close()

In [ ]:
# filef= open('Y_trainfull.pickle','rb')
# XER = pickle.load(filef)

In [ ]:
# len(XER)

In [ ]:
# XER

In [ ]:
# len(X_train)

In [ ]:
def DefineBasicModel():
    model = Sequential()
    model.add(Conv2D(64,(4,4),input_shape = (9,9,1),activation = 'relu'))
    model.add(Conv2D(32,(2,2),activation = 'relu'))
    model.add(Conv2D(64,(2,2),activation = 'relu'))
    model.add(Conv2D(32,(2,2),activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [ ]:
FlairModel = DefineBasicModel()
FlairModel.load_weights('../input/trainedmodelweights/model_FLAIR (1).h5')

T1wModel = DefineBasicModel()
T1wModel.load_weights('../input/trainedmodelweights/model_T1w.h5')

T1wCEModel = DefineBasicModel()
T1wCEModel.load_weights('../input/trainedmodelweights/model_T1wCE (2).h5')

T2wModel = DefineBasicModel()
T2wModel.load_weights('../input/trainedmodelweights/model_T2w.h5')

def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

FinalPrediction=[]
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'
trainset=[]
trainlabel=[]
trainidt=[]
for i in tqdm(range(10)):
    FinalCount=0
    idt=labels.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:] 
    path=os.path.join(train_dir,idt2,'FLAIR')
    Count=0
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        Currimg = np.array([image])
        Pred = FlairModel.predict(Currimg) 
        if(np.argmax(Pred)==0):
            Count-=1
        else:
            Count+=1
#         trainset+=[image]
#         trainlabel+=[labels.loc[i,'MGMT_value']]
#         trainidt+=[idt]
    if(Count>0):
        FinalCount+=1
    else:
        FinalCount-=1
        
    
    path=os.path.join(train_dir,idt2,'T1w')
    Count=0
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        Currimg = np.array([image])
        Pred = T1wModel.predict(Currimg) 
        if(np.argmax(Pred)==0):
            Count-=1
        else:
            Count+=1
#         trainset+=[image]
#         trainlabel+=[labels.loc[i,'MGMT_value']]
#         trainidt+=[idt]
    if(Count>0):
        FinalCount+=1
    else:
        FinalCount-=1
    
    path=os.path.join(train_dir,idt2,'T1wCE')
    Count=0
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        Currimg = np.array([image])
        Pred = T1wCEModel.predict(Currimg) 
        if(np.argmax(Pred)==0):
            Count-=1
        else:
            Count+=1
#         trainset+=[image]
#         trainlabel+=[labels.loc[i,'MGMT_value']]
#         trainidt+=[idt]
    if(Count>0):
        FinalCount+=1
    else:
        FinalCount-=1
    
    path=os.path.join(train_dir,idt2,'T2w')
    Count=0
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im))
        img=cv2.resize(img,(9,9)) 
        image=img_to_array(img)
        image=image/255.0
        Currimg = np.array([image])
        Pred = T2wModel.predict(Currimg) 
        if(np.argmax(Pred)==0):
            Count-=1
        else:
            Count+=1
#         trainset+=[image]
#         trainlabel+=[labels.loc[i,'MGMT_value']]
#         trainidt+=[idt]
    if(Count>0):
        FinalCount+=1
    else:
        FinalCount-=1
    
#     path=os.path.join(train_dir,idt2,'T2w')
#     Count=0
#     for im in os.listdir(path):
#         img=load_dicom(os.path.join(path,im))
#         img=cv2.resize(img,(9,9)) 
#         image=img_to_array(img)
#         image=image/255.0
#         Currimg = np.array(image)
#         Pred = T2wModel.predict([Currimg]) 
#         if(np.argmax(Pred[i])==0):
#             Count-=1
#         else:
#             Count+=1
# #         trainset+=[image]
# #         trainlabel+=[labels.loc[i,'MGMT_value']]
# #         trainidt+=[idt]
#     if(Count>0):
#         FinalCount+=1
#     else:
#         FinalCount-=1

    if(FinalCount>0):
        FinalPrediction.append(1)
    else:
        FinalPrediction.append(0)


In [ ]:
FinalPrediction

In [ ]:
90%